# Reto SAS: Fundraising Scenario Primera Parte

Edgar Castillo Ramírez A00827826

En esta primera parte lo que se realiza es que con ayuda de un modelo de Regresión se predice la cantidad que darán los donantes que fueron o no contactados. El modelo ganador fue Random Forest Model.

## Primera parte: Preparación de datos

**0- Connect to your Google Drive folder**

In [ ]:
my_folder = "/content/drive/MyDrive/2022/AI/A00827826_Fundraising"

from google.colab import drive
drive.mount('/content/drive')

# Change the following code to set your Drive folder
import os
os.chdir(my_folder)
!pwd

Mounted at /content/drive
/content/drive/MyDrive/2022/AI/A00827826_Fundraising


**1- Make sure that your Python version is 3.3 or higher as well as your Java version is 1.8.0_162 or higher**

In [ ]:
!echo "Python is at" $(which python)
!python --version

Python is at /usr/local/bin/python
Python 3.8.15


In [ ]:
!echo "Java is at" $(which java)
!/usr/bin/java -version

Java is at /usr/bin/java
openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)


**2- Install SASPy**

In [ ]:
!pip install saspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 9.4 MB/s 
  Created wheel for saspy: filename=saspy-4.4.1-py3-none-any.whl size=9937304 sha256=eedcc428b8d86ffce7c389d07dee6ef7af12d18cb447cf6b8fa212812db9fea7
  Stored in directory: /root/.cache/pip/wheels/0a/f6/ab/f7173c8a824856c66f6fd50194bb8fd3ef93fb4f1824be8d0c
Successfully built saspy


**3- Create the configuration file "sascfg_personal.py"**
Please, check that your Home Region is correct, you can check it at [ODA-SAS](https://welcome.oda.sas.com/home)

In [ ]:
%%writefile sascfg_personal.py
SAS_config_names=['oda']
oda = {'java' : '/usr/bin/java',
#US Home Region 1
#'iomhost' : ['odaws01-usw2.oda.sas.com','odaws02-usw2.oda.sas.com','odaws03-usw2.oda.sas.com','odaws04-usw2.oda.sas.com'],
#US Home Region 2
'iomhost' : ['odaws01-usw2-2.oda.sas.com','odaws02-usw2-2.oda.sas.com'],
#European Home Region 1
#'iomhost' : ['odaws01-euw1.oda.sas.com','odaws02-euw1.oda.sas.com'],
#Asia Pacific Home Region 1
#'iomhost' : ['odaws01-apse1.oda.sas.com','odaws02-apse1.oda.sas.com'],
#Asia Pacific Home Region 2
#'iomhost' : ['odaws01-apse1-2.oda.sas.com','odaws02-apse1-2.oda.sas.com'],
'iomport' : 8591,
'authkey' : 'oda',
'encoding' : 'utf-8'
}

Overwriting sascfg_personal.py


**4- Create your .authinfo**

In [ ]:
#%%writefile .authinfo
#oda user A00827826@tec.mx password 0683D0ctorJAY$

Writing .authinfo


Copy this file to home

In [ ]:
!cp .authinfo ~/.authinfo

**5- Establish Connection (Need to do this step each time you use SASPy)**

In [ ]:
import saspy
sas_session = saspy.SASsession(cfgfile=os.path.join(
    my_folder,"sascfg_personal.py"))
sas_session

Using SAS Config named: oda
SAS Connection established. Subprocess id is 356



Access Method         = IOM
SAS Config name       = oda
SAS Config file       = /content/drive/MyDrive/2022/AI/A00827826_Fundraising/sascfg_personal.py
WORK Path             = /saswork/SAS_work2FD900012138_odaws02-usw2-2.oda.sas.com/SAS_work2EA800012138_odaws02-usw2-2.oda.sas.com/
SAS Version           = 9.04.01M6P11072018
SASPy Version         = 4.4.1
Teach me SAS          = False
Batch                 = False
Results               = Pandas
SAS Session Encoding  = utf-8
Python Encoding value = utf-8
SAS process Pid value = 74040


**SAS Dataset Connection**

In [ ]:
ps = sas_session.submit("""
    libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
    """)
print(ps["LOG"])


5                                                          The SAS System                      Friday, December  2, 2022 06:04:00 AM

24         ods listing close;ods html5 (id=saspy_internal) file=_tomods1 options(bitmap_mode='inline') device=svg style=HTMLBlue;
24       ! ods graphics on / outputfmt=png;
25         
26         
27             libname cortex '~/my_shared_file_links/u39842936/Cortex Data Sets';
28         
29         
30         
31         ods html5 (id=saspy_internal) close;ods listing;
32         

6                                                          The SAS System                      Friday, December  2, 2022 06:04:00 AM

33         


**Dataset conversion into Pandas**

In [ ]:
import pandas as pd

#comment: Transform cloud sas dataset to python dataframe(pandas) ==> might take some time.

data1 = sas_session.sasdata2dataframe(
table='hist',
libref='cortex'
)

data2 = sas_session.sasdata2dataframe(
table='target_rd2',
libref='cortex'
)

**Merging the data**

In [ ]:
data_merge = pd.merge(data1, data2, on=["ID"],how="right")
data_merge = data_merge.loc[(data_merge['GaveThisYear'] ==1)]
data_merge.sample(2)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
841898,2841899.0,WALSH,DANIELLE,1.0,47.0,62400.0,University / College,City,10.0,1.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,40.0
292628,2292629.0,CRAIG,WILLIE,1.0,37.0,39700.0,High School,Suburban,6.0,1.0,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,100.0


**Missing values**

In [ ]:
data_merge.describe()

,ID,Woman,Age,Salary,SeniorList,NbActivities,Referrals,Recency,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
count,1.494570e+05,149457.000000,149457.000000,149457.000000,149457.000000,149457.000000,149457.000000,68433.000000,68433.000000,68433.000000,68433.000000,68433.000000,68433.000000,149457.000000,149457.000000,149457.000000,149457.0,149457.000000
mean,2.499355e+06,0.544979,45.256642,63898.012137,4.415210,0.917361,0.911459,2.750910,1.985796,4.674952,134.864685,42.502521,106.674485,0.208207,14.167085,0.261286,1.0,62.959012
std,2.887997e+05,0.497974,18.915539,60194.539355,3.345957,1.334288,1.412152,1.956686,1.297829,2.613877,372.981373,160.658491,344.831631,0.406027,121.163666,0.439337,0.0,227.894674
min,2.000004e+06,0.000000,16.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,10.000000,10.000000,10.000000,0.000000,0.000000,0.000000,1.0,10.000000
25%,2.249026e+06,0.000000,29.000000,20300.000000,1.000000,0.000000,0.000000,1.000000,1.000000,2.000000,25.000000,10.000000,20.000000,0.000000,0.000000,0.000000,1.0,20.000000
50%,2.499486e+06,1.000000,44.000000,43000.000000,4.000000,0.000000,0.000000,2.000000,2.000000,4.000000,50.000000,20.000000,30.000000,0.000000,0.000000,0.000000,1.0,25.000000
75%,2.749383e+06,1.000000,59.000000,87000.000000,7.000000,1.000000,1.000000,4.000000,3.000000,7.000000,115.000000,30.000000,75.000000,0.000000,0.000000,1.000000,1.0,50.000000
max,2.999999e+06,1.000000,90.000000,250000.000000,10.000000,10.000000,15.000000,10.000000,10.000000,10.000000,12160.000000,10000.000000,10000.000000,1.000000,10000.000000,1.000000,1.0,10000.000000


In [ ]:
#Podemos ver que Recency, Frequency, Seniority, TotalGift, MinGift y MaxGift son variables que tienen una buena cantidad de NaN
data_merge.isnull().sum()

ID                  0
LastName            3
FirstName           0
Woman               0
Age                 0
Salary              0
Education           0
City                0
SeniorList          0
NbActivities        0
Referrals           0
Recency         81024
Frequency       81024
Seniority       81024
TotalGift       81024
MinGift         81024
MaxGift         81024
GaveLastYear        0
AmtLastYear         0
Contact             0
GaveThisYear        0
AmtThisYear         0
dtype: int64

In [ ]:
# Esto no se usa

data_merge[['MinGift']] = data_merge[['MinGift']].fillna(value=data_merge['MinGift'].mean())  
data_merge[['MaxGift']] = data_merge[['MaxGift']].fillna(value=data_merge['MaxGift'].mean())  

data_merge.sample(5)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
195516,2195517.0,MCAVOY,MELANIE,1.0,43.0,31300.0,University / College,Rural,6.0,2.0,...,2.0,5.0,170.0,20.000000,150.000000,0.0,0.0,1.0,1.0,40.0
159832,2159833.0,LOVE,MARTHA,1.0,70.0,19100.0,University / College,Downtown,4.0,2.0,...,3.0,3.0,115.0,10.000000,75.000000,1.0,20.0,0.0,1.0,20.0
489944,2489945.0,HICKS,JOHN,0.0,60.0,27100.0,High School,Downtown,6.0,2.0,...,1.0,5.0,25.0,25.000000,25.000000,0.0,0.0,0.0,1.0,25.0
750077,2750078.0,MCDONALD,GEORGE,0.0,64.0,97500.0,University / College,Suburban,6.0,1.0,...,NaN,NaN,NaN,42.502521,106.674485,0.0,0.0,0.0,1.0,100.0
283257,2283258.0,MACE,GERALD,0.0,66.0,22200.0,High School,Suburban,10.0,1.0,...,3.0,8.0,50.0,10.000000,20.000000,0.0,0.0,0.0,1.0,30.0


**Data Partition**

In [ ]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(data_merge, test_size=0.2,random_state=42)
train.sample(5)

,ID,LastName,FirstName,Woman,Age,Salary,Education,City,SeniorList,NbActivities,...,Frequency,Seniority,TotalGift,MinGift,MaxGift,GaveLastYear,AmtLastYear,Contact,GaveThisYear,AmtThisYear
465765,2465766.0,LIND,ERIKA,1.0,39.0,67100.0,University / College,Suburban,6.0,3.0,...,3.0,6.0,70.0,20.000000,30.000000,1.0,25.0,0.0,1.0,50.0
739864,2739865.0,GARDNER,CHRISTINA,1.0,33.0,58800.0,University / College,City,0.0,0.0,...,NaN,NaN,NaN,42.502521,106.674485,0.0,0.0,1.0,1.0,25.0
189302,2189303.0,MACLEAN,ANITA,1.0,28.0,109700.0,University / College,Downtown,9.0,0.0,...,NaN,NaN,NaN,42.502521,106.674485,0.0,0.0,1.0,1.0,30.0
584313,2584314.0,HEADLEY,KRISTY,1.0,73.0,4800.0,High School,City,4.0,1.0,...,2.0,4.0,60.0,30.000000,30.000000,0.0,0.0,0.0,1.0,25.0
333401,2333402.0,GUADALUPE,KAREN,1.0,50.0,178200.0,University / College,Downtown,4.0,1.0,...,NaN,NaN,NaN,42.502521,106.674485,1.0,50.0,1.0,1.0,10.0


## Segunda parte: Modelos

**Linear Regression Model**

In [ ]:
from sklearn import linear_model
import numpy as np
from sklearn import metrics
X_train = train[['Age', 'Salary','Contact', 'GaveLastYear','Woman', 'NbActivities', 'Referrals' ]] 
Y_train = train['AmtThisYear']
X_valid = validation[['Age', 'Salary','Contact', 'GaveLastYear','Woman', 'NbActivities', 'Referrals' ]] 
Y_valid = validation['AmtThisYear']

regr = linear_model.LinearRegression()

regr.fit(X_train,Y_train)

regr_predict_Train=regr.predict(X_train)
regr_predict_Valid=regr.predict(X_valid)

print(metrics.mean_squared_error(Y_train,regr_predict_Train))
print(metrics.r2_score(Y_train, regr_predict_Train))

print(metrics.mean_squared_error(Y_valid,regr_predict_Valid)) 
print(metrics.r2_score(Y_valid, regr_predict_Valid))

51415.341965101536
0.008849919326195943
51679.689505725975
0.009591375384806589


**Lasso Model**

In [ ]:
from sklearn.linear_model import Lasso

model_lasso = Lasso(alpha=0.01)
model_lasso.fit(X_train,Y_train) 

lasso_predict_Train=model_lasso.predict(X_train)
lasso_predict_Valid=model_lasso.predict(X_valid)

print(metrics.mean_squared_error(Y_train,lasso_predict_Train))
print(metrics.r2_score(Y_train, lasso_predict_Train))

print(metrics.mean_squared_error(Y_valid,lasso_predict_Valid)) 
print(metrics.r2_score(Y_valid, lasso_predict_Valid))

51414.10797906982
0.00887370727174186
51675.12500626652
0.009678851135433764


**Ridge Model**

In [ ]:
from sklearn.linear_model import Ridge

model_ridge = Ridge(alpha=0.01)
model_ridge.fit(X_train,Y_train) 

ridge_predict_Train=model_ridge.predict(X_train)
ridge_predict_Valid=model_ridge.predict(X_valid)

print(metrics.mean_squared_error(Y_train,ridge_predict_Train))
print(metrics.r2_score(Y_train, ridge_predict_Train))

print(metrics.mean_squared_error(Y_valid,ridge_predict_Valid)) 
print(metrics.r2_score(Y_valid, ridge_predict_Valid))

51414.1063464841
0.00887373874362174
51675.33313886877
0.009674862405503193


**Decision Tree Model**

In [ ]:
from sklearn.tree import DecisionTreeRegressor

DT_model = DecisionTreeRegressor(max_depth=5, random_state=42).fit(X_train,Y_train)
DT_predict_Train=DT_model.predict(X_train)
DT_predict_Valid=DT_model.predict(X_valid)

print(metrics.mean_squared_error(Y_train,DT_predict_Train))
print(metrics.r2_score(Y_train, DT_predict_Train))

print(metrics.mean_squared_error(Y_valid,DT_predict_Valid)) 
print(metrics.r2_score(Y_valid, DT_predict_Valid))

49940.54288041204
0.037280095532352786
51299.48191885439
0.0168778137667559


**Random Forest Model**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RFR_model = RandomForestRegressor(n_estimators= 300, max_depth= 5)
RFR_model.fit(X_train,Y_train)

RFR_predict_Train=RFR_model.predict(X_train)
RFR_predict_Valid=RFR_model.predict(X_valid)

print(metrics.mean_squared_error(Y_train,RFR_predict_Train))
print(metrics.r2_score(Y_train, RFR_predict_Train))

print(metrics.mean_squared_error(Y_valid,RFR_predict_Valid)) 
print(metrics.r2_score(Y_valid, RFR_predict_Valid))


49641.08673156914
0.04305280800960065
51200.326933941826
0.01877805645726427


**XGBoost Model**

In [ ]:
import xgboost as xg

XGB_model =  xg.XGBRegressor(n_estimators=50).fit(X_train, Y_train)
XGB_predict_Train = XGB_model.predict(X_train)
XGB_predict_Valid = XGB_model.predict(X_valid)

print(metrics.mean_squared_error(Y_train,XGB_predict_Train))
print(metrics.r2_score(Y_train, XGB_predict_Train))

print(metrics.mean_squared_error(Y_valid,XGB_predict_Valid)) 
print(metrics.r2_score(Y_valid, XGB_predict_Valid))

[16:15:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
50234.810274364165
0.03160740835168041
51196.83243418638
0.018845026339041948


## Tercera parte: Scoring y Creación del archivo CSV

**Prepare the data**

In [ ]:
data3 = sas_session.sasdata2dataframe(
table='score',
libref='cortex'
)
data4 = sas_session.sasdata2dataframe(
table='score_rd2_contact',
libref='cortex'
)
data5 = sas_session.sasdata2dataframe(
table='score_rd2_nocontact',
libref='cortex'
)

**Scoring the data with the best model found**

### Predict 'amount given' for members who were contacted

In [ ]:
scoring_data_contact = pd.merge(data3, data4, on=["ID"],how="right")

X = scoring_data_contact[['Age', 'Salary','Contact', 'GaveLastYear','Woman', 'NbActivities', 'Referrals' ]] 

XGB_predict_contact=RFR_model.predict(X)

scoring_data_contact['Prediction'] = XGB_predict_contact

scoring_data_contact= scoring_data_contact[['ID','Prediction']]
scoring_data_contact = scoring_data_contact.rename({'Prediction': 'AmtContact'}, axis=1) 
scoring_data_contact.head()

,ID,AmtContact
0,2000001.0,57.830584
1,2000002.0,59.977259
2,2000003.0,96.433010
3,2000004.0,42.711416
4,2000005.0,103.531904


### Predict 'amount given' for members who were not contacted

In [ ]:
scoring_data_nocontact = pd.merge(data3, data5, on=["ID"],how="right")

X = scoring_data_nocontact[['Age', 'Salary','Contact', 'GaveLastYear','Woman', 'NbActivities', 'Referrals' ]] 

XGB_predict_nocontact=RFR_model.predict(X)

scoring_data_nocontact['Prediction'] = XGB_predict_nocontact

scoring_data_nocontact= scoring_data_nocontact[['ID','Prediction']]
scoring_data_nocontact = scoring_data_nocontact.rename({'Prediction': 'AmtNoContact'}, axis=1) 
scoring_data_nocontact.head()

,ID,AmtNoContact
0,2000001.0,57.942645
1,2000002.0,60.045526
2,2000003.0,96.571471
3,2000004.0,42.878443
4,2000005.0,103.670365


In [ ]:
result_Amt = pd.merge(scoring_data_contact, scoring_data_nocontact, on=["ID"],how="right")
result_Amt.sort_values(by=['ID'], inplace=True)
result_Amt.head(3)

,ID,AmtContact,AmtNoContact
0,2000001.0,57.830584,57.942645
1,2000002.0,59.977259,60.045526
2,2000003.0,96.433010,96.571471


### **CSV Files**

In [ ]:
result_Amt.to_csv('Round2_Output_amt.csv', index=False)

In [ ]:
!head Round2_Output_amt.csv

ID,AmtContact,AmtNoContact
2000001.0,57.830583765215245,57.942644804777146
2000002.0,59.97725938707491,60.04552616545727
2000003.0,96.43300976607212,96.57147114216163
2000004.0,42.711416401857655,42.87844289893083
2000005.0,103.53190373804136,103.67036511413087
2000006.0,37.3799797057248,36.21242674461002
2000007.0,29.339599781012826,29.384994084375037
2000008.0,57.84070038141309,57.95276142097499
2000009.0,44.92718970530973,44.92718970530973
